In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from category_encoders import OneHotEncoder, TargetEncoder
from scipy import stats
import statsmodels.api as sm

# Cargar los conjuntos de datos
ruta_train = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/traincase.csv'
ruta_test = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/testcase.csv'
df_train = pd.read_csv(ruta_train)
df_test = pd.read_csv(ruta_test)

# Función para preprocesar y limpiar el DataFrame
def preprocess_df(df):
    # Convertir y limpiar columnas específicas
    for col in ['Search Engine Bid', 'Avg. Pos.', 'Avg. Cost per Click', 'Impressions']:
        if df[col].dtype == object:  # Aplicar operaciones de string solo si la columna es de tipo object
            df[col] = pd.to_numeric(df[col].str.replace('$', '').str.replace(',', '').str.strip(), errors='coerce')
    
    # Llenar valores faltantes para 'Impressions' después de la conversión
    df['Impressions'].fillna(df['Impressions'].median(), inplace=True)

    # TF-IDF para 'Keyword'
    tfidf_vectorizer = TfidfVectorizer(max_features=600)
    # Transformar 'Keyword' y ajustar k-means en la representación TF-IDF
    keywords_tfidf = tfidf_vectorizer.fit_transform(df['Keyword'].str.lower())
    kmeans = KMeans(n_clusters=10, random_state=42)
    keyword_clusters = kmeans.fit_predict(keywords_tfidf)
    df['Keyword Cluster'] = keyword_clusters
    
    # Crear características de interacción
    df['Interaction'] = df['Keyword'] + '_' + df['Match Type']
    
    # Binning para 'Impressions Category'
    if 'Impressions Category' not in df.columns:
        bin_edges = [0, 100, 1000, 10000, np.inf]
        bin_labels = [1, 2, 3, 4]
        df['Impressions Category'] = pd.cut(df['Impressions'], bins=bin_edges, labels=bin_labels, right=False).cat.add_categories([0]).fillna(0).astype(int)
    
    return df

# Suponiendo que df_train y df_test ya están cargados
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

# Continuar con la selección de características y la preparación de los datos para el modelado
selected_features = ['Search Engine Bid', 'Impressions Category', 'Avg. Pos.', 'Keyword Cluster']
X_train = df_train[selected_features]
y_train = df_train['Clicks']

# Asegurar que todas las columnas son numéricas
X_train = X_train.apply(pd.to_numeric, errors='coerce')

# Eliminar filas con valores NaN en X_train y y_train
X_train.dropna(inplace=True)
y_train = y_train.loc[X_train.index]

# Ajustar el modelo
X_train_with_const = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train_with_const.astype(float))
results = model.fit()

# Mostrar el resumen del modelo
print(results.summary())







ValueError: could not convert string to float: '29,060'

In [8]:
# Guarda 'entry_id' del conjunto de prueba para usarlo en la exportación de predicciones
entry_id_test = df_test['entry_id'].copy()

# Asegúrate de que X_train y X_test no incluyan 'entry_id'
X_train = df_train[selected_features]
X_test = df_test[selected_features]
y_train = df_train['Clicks']

# Definir el preprocesador para el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), ['Search Engine Bid', 'Avg. Pos.', 'Avg. Cost per Click']),
        ('cat', OneHotEncoder(), ['Impressions Category', 'Keyword Cluster'])
    ],
    remainder='passthrough'  # Mantener el resto de las columnas sin cambios
)

# Definir el pipeline del modelo
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador con ColumnTransformer
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))  # Modelo de regresión aleatoria
])

# Entrenar el modelo
model_pipeline.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
predictions = model_pipeline.predict(X_test)

# Crear DataFrame para exportación que incluya 'entry_id' y las predicciones
predictions_df = pd.DataFrame({'entry_id': entry_id_test, 'Prediction': predictions})

# Exportar las predicciones a un archivo CSV
predictions_df.to_csv('predictions6.csv', index=False)






ValueError: A given column is not a column of the dataframe